In [10]:
import pandas as pd
from google.cloud import bigquery

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

/home/sebastien/.pyenv/versions/3.10.6/envs/NLP-Systematic-review/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/sebastien/.pyenv/versions/3.10.6/envs/NLP-Systematic-review/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/se

In [2]:
# Variable
GCP_PROJECT_SEBT84='useful-proposal-392710'
GCP_REGION='europe-west1'

# BigQuery
BQ_REGION='EU'
BQ_DATASET='enhancing_systemic_reviews_nlp'
TABLE = 'pubmed'

# Cloud Storage
BUCKET_NAME='enhancing_systemic_reviews_nlp'

In [6]:
df = pd.read_csv('/home/sebastien/code/CuevasEli/NLP-Systematic-review/raw_data/data.csv')

#size = 2000
df_size = df.shape[0]
frac = 0.02
sample_size = round(df_size*frac)
display(f'sample size: {sample_size}')

df_small = df.iloc[0:sample_size]

df_train = df_small['abstract_text']
#df_train.shape

'sample size: 3813'

# Baseline Model

In [11]:
topic_model = BERTopic()

topics, probs = topic_model.fit_transform(df_train)

In [24]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,885,-1_the_of_and_in,"[the, of, and, in, to, with, were, was, patien...",[Treatment of acute Kawasaki syndrome with a f...
1,0,126,0_asthma_inhaled_fev1_salbutamol,"[asthma, inhaled, fev1, salbutamol, albuterol,...",[The objective of this trial was to determine ...
2,1,124,1_anesthesia_intubation_isoflurane_mgkg,"[anesthesia, intubation, isoflurane, mgkg, pro...",[To determine the effectiveness of esmolol hyd...
3,2,121,2_pressure_blood_diastolic_hg,"[pressure, blood, diastolic, hg, mm, hypertens...",[Dose-response evaluation of oral labetalol ( ...
4,3,81,3_insulin_glucose_diabetes_control,"[insulin, glucose, diabetes, control, niddm, g...",[To compare the effect of bedtime NPH insulin ...
...,...,...,...,...,...
96,95,13,95_renal_creatinine_plasmapheresis_lupus,"[renal, creatinine, plasmapheresis, lupus, pre...",[To determine if deterioration in renal functi...
97,96,12,96_ondansetron_granisetron_nausea_emetic,"[ondansetron, granisetron, nausea, emetic, che...",[To determine the contribution of dexamethason...
98,97,11,97_warts_ifn_cryotherapy_interferon,"[warts, ifn, cryotherapy, interferon, hpv, ag,...",[Genital warts are a highly prevalent and chro...
99,98,11,98_intervention_prenatal_home_postpartum,"[intervention, prenatal, home, postpartum, chi...",[The goal of our study was to measure the effe...


In [17]:
distance_map = topic_model.visualize_topics()
distance_map.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/baseline_distance_map.html')

matrix = topic_model.visualize_heatmap()
matrix.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/baseline_matrix.html')

bar_chart = topic_model.visualize_barchart()
bar_chart.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/baseline_bar_chart.html')

rank = topic_model.visualize_term_rank(log_scale=True)
rank.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/baseline_rank.html')

# Optimisation with less groups

In [33]:
from umap import UMAP
from hdbscan import HDBSCAN

from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [29]:
#umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=25, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

topic_model = BERTopic(hdbscan_model=hdbscan_model
                        , top_n_words=10
                        , n_gram_range =(1, 2)
                        , verbose=True )

topics, probs = topic_model.fit_transform(df_train)

Batches:   0%|          | 0/120 [00:00<?, ?it/s]

2023-09-07 17:22:48,364 - BERTopic - Transformed documents to Embeddings
2023-09-07 17:23:06,376 - BERTopic - Reduced dimensionality
2023-09-07 17:23:06,525 - BERTopic - Clustered reduced embeddings


In [38]:
print(f"Model coherence:{coherence_metric(topic_model,df_small)}")
topic_model.get_topic_info()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Topic,Count,Name,Representation,Representative_Docs
0,-1,515,-1_the_of_and_in,"[the, of, and, in, to, were, with, patients, w...",[To evaluate the efficacy of annual mammograph...
1,0,730,0_the_in_of_and,"[the, in, of, and, patients, to, with, was, bl...",[The aim of the present study was to evaluate ...
2,1,321,1_the_of_and_to,"[the, of, and, to, were, in, for, patients, wi...",[To improve end-of-life decision making and re...
3,2,283,2_patients_survival_of_and,"[patients, survival, of, and, chemotherapy, th...",[To determine the effect of concurrent chest r...
4,3,189,3_asthma_of_in_the,"[asthma, of, in, the, and, to, with, fev1, inh...","[An effective , long-acting bronchodilator cou..."
5,4,181,4_the_and_anesthesia_of,"[the, and, anesthesia, of, to, was, in, with, ...",[To determine the effectiveness of esmolol hyd...
6,5,176,5_women_the_of_delivery,"[women, the, of, delivery, in, labor, fetal, t...",[Beta-adrenergic agonists are commonly used to...
7,6,159,6_of_patients_and_the,"[of, patients, and, the, in, with, were, infec...",[In patients with the acquired immunodeficienc...
8,7,133,7_pain_bupivacaine_analgesia_epidural,"[pain, bupivacaine, analgesia, epidural, the, ...",[The use of neuraxial opioids has gained popul...
9,8,104,8_cholesterol_lipoprotein_levels_and,"[cholesterol, lipoprotein, levels, and, in, th...",[This study was designed to evaluate the effec...


In [31]:
distance_map = topic_model.visualize_topics()
distance_map.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/topic_size_limit_distance_map.html')

matrix = topic_model.visualize_heatmap()
matrix.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/topic_size_limit_matrix.html')

bar_chart = topic_model.visualize_barchart()
bar_chart.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/topic_size_limit_bar_chart.html')

rank = topic_model.visualize_term_rank(log_scale=True)
rank.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/topic_size_limit_rank.html')

In [32]:
def coherence_metric(topic_model, data):
    """
    Calculate coherence score for topics generated by BERTopic.

    Parameters:
    - topic_model: Trained BERTopic model
    - df: DataFrame containing the text data

    Returns:
    - coherence: Average topic coherence score
    """

    topics = topic_model.get_topics()


    texts = [text.split() for text in data['abstract_text']]
    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]


    gensim_topics = {key: [word[0] for word in value] for key, value in topics.items()}
    lda_topics = list(gensim_topics.values())


    cm = CoherenceModel(topics=lda_topics, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence = cm.get_coherence()

    return coherence

# Optimisation: vectorising the document

In [42]:
from nltk.tokenize import sent_tokenize, word_tokenize
#sentences = [sent_tokenize(abstract) for abstract in abstracts]
#sentences = [sentence for doc in sentences for sentence in doc]

In [48]:
df_train
sentences = [sent_tokenize(abstract) for abstract in df_train]
#sentences = [sentence for doc in sentences for sentence in doc]

In [56]:
vectorised_model = BERTopic(verbose=True)

topics, probs = vectorised_model.fit_transform(sentences)

TypeError: Make sure that the iterable only contains strings.

In [54]:
print(f"Model coherence:{coherence_metric(vectorised_model,df_small)}")
vectorised_model.get_topic_info()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Topic,Count,Name,Representation,Representative_Docs
0,-1,787,-1_the_of_and_in,"[the, of, and, in, to, patients, with, were, w...",[Treatment of acute Kawasaki syndrome with a f...
1,0,192,0_women_delivery_labor_fetal,"[women, delivery, labor, fetal, magnesium, pre...",[Our purpose was to determine whether seizure ...
2,1,128,1_asthma_fev1_inhaled_salbutamol,"[asthma, fev1, inhaled, salbutamol, expiratory...",[The objective of this trial was to determine ...
3,2,122,2_anesthesia_intubation_isoflurane_mgkg,"[anesthesia, intubation, isoflurane, mgkg, pro...",[To determine if a total intravenous ( i.v. ) ...
4,3,92,3_cholesterol_lipoprotein_levels_pravastatin,"[cholesterol, lipoprotein, levels, pravastatin...",[This study was designed to evaluate the effec...
...,...,...,...,...,...
92,91,13,91_residents_students_instruction_lecture,"[residents, students, instruction, lecture, pa...",[Laboratory investigations are inappropriately...
93,92,12,92_ondansetron_granisetron_nausea_emetic,"[ondansetron, granisetron, nausea, emetic, che...",[To determine the contribution of dexamethason...
94,93,11,93_sleep_flumazenil_triazolam_dawn,"[sleep, flumazenil, triazolam, dawn, benzodiaz...",[Thirty-one patients with acute or chronic liv...
95,94,11,94_catheter_catheters_venous_insertion,"[catheter, catheters, venous, insertion, cathe...",[To identify risk factors for infusion-related...


In [55]:
distance_map = vectorised_model.visualize_topics()
distance_map.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/vectorised_model_limit_distance_map.html')

matrix = vectorised_model.visualize_heatmap()
matrix.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/vectorised_model_limit_matrix.html')

bar_chart = vectorised_model.visualize_barchart()
bar_chart.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/vectorised_model_limit_bar_chart.html')

rank = vectorised_model.visualize_term_rank(log_scale=True)
rank.write_html('/home/sebastien/code/CuevasEli/NLP-Systematic-review/fig/vectorised_model_limit_rank.html')

# Optimisation: vectorising the document + limitation of topic size